## Deliverable 2. Create a Customer Travel Destinations Map.

In [33]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [34]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Condition
0,0,Nanortalik,60.1432,-45.2371,35.17,82,88,8.19,GL,overcast clouds
1,1,Arraial Do Cabo,-22.9661,-42.0278,68.92,89,100,17.52,BR,overcast clouds
2,2,College,64.8569,-147.8028,19.72,90,90,4.61,US,overcast clouds
3,3,Norman Wells,65.2820,-126.8329,24.84,93,90,2.30,CA,light snow
4,4,Ribeira Grande,38.5167,-28.7000,69.53,90,81,13.73,PT,broken clouds


In [35]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 88


In [36]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Condition
5,5,Waingapu,-9.6567,120.2641,87.67,62,3,8.12,ID,clear sky
7,7,Bonthe,7.5264,-12.5050,76.77,87,100,7.16,SL,moderate rain
9,9,Tecoanapa,16.5167,-98.7500,80.47,82,6,6.71,MX,clear sky
12,12,Itoman,26.1247,127.6694,80.62,57,75,4.61,JP,broken clouds
13,13,Carlagan,14.9750,122.0456,81.37,76,83,15.86,PH,broken clouds
14,14,Butaritari,3.0707,172.7902,83.28,68,4,16.20,KI,clear sky
21,21,Kapaa,22.0752,-159.3190,80.58,83,20,3.00,US,few clouds
22,22,Rikitea,-23.1203,-134.9692,75.54,81,100,15.68,PF,overcast clouds
24,24,Kahului,20.8947,-156.4700,84.22,68,20,16.11,US,few clouds
28,28,Jutai,-5.1833,-68.9000,77.20,79,89,1.23,BR,overcast clouds


In [37]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0           173
City                 173
Lat                  173
Lng                  173
Max Temp             173
Humidity             173
Cloudiness           173
Wind Speed           173
Country              172
Weather Condition    173
dtype: int64

In [52]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_rev_df = preferred_cities_df.dropna(how = 'all')
preferred_cities_rev_df.count()

Unnamed: 0           173
City                 173
Lat                  173
Lng                  173
Max Temp             173
Humidity             173
Cloudiness           173
Wind Speed           173
Country              172
Weather Condition    173
dtype: int64

In [53]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_rev_df[["City", "Country", "Max Temp", "Weather Condition", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Condition,Lat,Lng,Hotel Name
5,Waingapu,ID,87.67,clear sky,-9.6567,120.2641,
7,Bonthe,SL,76.77,moderate rain,7.5264,-12.5050,
9,Tecoanapa,MX,80.47,clear sky,16.5167,-98.7500,
12,Itoman,JP,80.62,broken clouds,26.1247,127.6694,
13,Carlagan,PH,81.37,broken clouds,14.9750,122.0456,
14,Butaritari,KI,83.28,clear sky,3.0707,172.7902,
21,Kapaa,US,80.58,few clouds,22.0752,-159.3190,
22,Rikitea,PF,75.54,overcast clouds,-23.1203,-134.9692,
24,Kahului,US,84.22,few clouds,20.8947,-156.4700,
28,Jutai,BR,77.20,overcast clouds,-5.1833,-68.9000,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

In [55]:
# Get hotel count to check if there are any without a name
hotel_df.count()


City                 173
Country              172
Max Temp             173
Weather Condition    173
Lat                  173
Lng                  173
Hotel Name           173
dtype: int64

In [56]:
# 7A. Drop the rows where there is no Hotel Name
hotel_rev_df = hotel_df.dropna(how = 'any')
hotel_rev_df.count()

City                 172
Country              172
Max Temp             172
Weather Condition    172
Lat                  172
Lng                  172
Hotel Name           172
dtype: int64

In [44]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_rev_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
